In [ ]:
#for information about theory see PHYSICAL REVIEW APPLIED 15, 064053 (2021)

import numpy as np
import matplotlib.pyplot as plt

kmesh = 40000

u  = np.zeros((kmesh,48))
en = np.zeros((kmesh,48))

      
with open('geninterp.dat') as fp:
    for i, line in enumerate(fp):
        j = i - 48*(i//48)
        item = line.split()     
        en[int(item[0])-1][j] = float(item[4])
        u[int(item[0])-1][j]  = float(item[5])         
        
              
#SI units        
# eV to J        
en = en*1.60217662e-19

#eV*ang to m/c  dE/d(hbar k)
u = u*(1.60217662e-29)/(1.054571e-34) 

# K to J
T = 300*1.3806e-23

# square of unit cell from A^2 to m^2
S = 13.8564064605*(1e-20)


# electron charge in Coul
e_charge = 1.60217662e-19


#Plank constant (J c)
hbar = 1.054571e-34

#Plank constant (eV c)
hbar_eV = 6.582e-16


#dielectric vacuum constant (F/m)
eps = 8.854187e-12


#effective height (m)
h_slab = 5.21872-10  


#speed of light (m/c)
c_speed  = 299792458

#refractive index of SiO2 substrate
n_subs = 1.54919333848297

print('JOB DONE!')

In [ ]:
energy  = np.zeros(352)
sigma_inter  = np.zeros(352, dtype=complex)
sigma_total  = np.zeros(352, dtype=complex)
sigma_A  = np.zeros(352, dtype=complex)


with open('kubo_S_xx.dat') as fp:
    for i, line in enumerate(fp):
        item = line.split()     
        energy[i] = float(item[0])
        sigma_inter[i] = complex(float(item[1]), float(item[2])) *26.2e-8 # from Sm/sm to Sm
        

with open('kubo_A_xy.dat') as fp:
    for i, line in enumerate(fp):
        item = line.split()     
        sigma_A[i] = complex(float(item[1]), float(item[2])) *26.2e-8 # from Sm/sm to Sm        
        

omega = 0.0
for k in range(kmesh):
    for n in range(48):
        omega+= (e_charge**2 * hbar)/(S * kmesh) * ((np.exp(en[k][n]/T)/T)/(1 + np.exp(en[k][n]/T))**2)*(u[k][n]**2)        
        
#calculate interband and intraband terms of conductivity
for i in range(352):
    sigma_intra =  1/(0.01*1.60217662e-19 - 1j * energy[i]*1.60217662e-19) * omega # in Sm    
    sigma_total[i] = (sigma_intra + sigma_inter[i])# /7.748e-5 #convert to conductance quantum 
    
print('JOB DONE!')

In [ ]:
Psi = np.zeros(352, dtype=complex)

for i in range(352):
    epsilon   = 1 + 1j*sigma_total[i]/(h_slab * (energy[i]/hbar_eV) * eps)
    epsilon_A = 1j*sigma_A[i]/(h_slab * (energy[i]/hbar_eV) * eps)
    
    Q = 1j*epsilon_A/epsilon
    
    Psi[i] = 2 * n_subs * Q * h_slab * (energy[i]/hbar_eV) / (c_speed *(n_subs**2  - 1))*1000
    

#Kerr rotation and  Kerr ellipticity  (mrad)
for i in range(352):
    print(energy[i], '\t', (Psi[i]).real, '\t', -(Psi[i]).imag)   